In [18]:
import json
import pandas as pd
import pymongo
from pymongo import MongoClient

# Connect to MongoDB
mongo_cl = MongoClient("mongodb://localhost:27017/")
mongo_db = mongo_cl["wa_air_quality"]
mongo_collection = mongo_db["daily_summary"]


In [20]:
# Load JSON
with open("air_quality_washington.json") as f:
    data = json.load(f)


In [22]:
# Print first 2 records to inspect structure
print(data[:2])


[{'state_code': '53', 'county_code': '033', 'site_number': '0030', 'parameter_code': '88101', 'poc': 5, 'latitude': 47.597222, 'longitude': -122.319722, 'datum': 'WGS84', 'parameter': 'PM2.5 - Local Conditions', 'sample_duration_code': '1', 'sample_duration': '1 HOUR', 'pollutant_standard': None, 'date_local': '2024-01-01', 'units_of_measure': 'Micrograms/cubic meter (LC)', 'event_type': 'No Events', 'observation_count': 24, 'observation_percent': 100.0, 'validity_indicator': 'Y', 'arithmetic_mean': 20.083333, 'first_max_value': 43.0, 'first_max_hour': 6, 'aqi': None, 'method_code': '170', 'method': 'Met One BAM-1020 Mass Monitor w/VSCC - Beta Attenuation', 'local_site_name': 'Seattle-10th & Weller', 'site_address': '10th & Weller', 'state': 'Washington', 'county': 'King', 'city': 'Seattle', 'cbsa_code': '42660', 'cbsa': 'Seattle-Tacoma-Bellevue, WA', 'date_of_last_change': '2025-04-04'}, {'state_code': '53', 'county_code': '033', 'site_number': '0030', 'parameter_code': '88101', 'poc'

In [24]:
#Inserting data in mongo collection
mongo_collection.insert_many(data)
print("Data uploaded successfully!")


Data uploaded successfully!


In [26]:
# Total number of docs present in collection
count = mongo_collection.count_documents({}) # count docs in collection
print(f"Total documents: {count}")

Total documents: 70939


## Data Transformation

In [30]:
import pandas as pd

df = mongo_collection.find() # gathering all docs from mongo

mongo_data = pd.DataFrame(list(df)) # converting the data into dataframe

print(mongo_data.dtypes) # checking the datatypes of each column

print(mongo_data.isnull().sum()) # checking if any value is missing in columns



_id                      object
state_code               object
county_code              object
site_number              object
parameter_code           object
poc                       int64
latitude                float64
longitude               float64
datum                    object
parameter                object
sample_duration_code     object
sample_duration          object
pollutant_standard       object
date_local               object
units_of_measure         object
event_type               object
observation_count         int64
observation_percent     float64
validity_indicator       object
arithmetic_mean         float64
first_max_value         float64
first_max_hour            int64
aqi                     float64
method_code              object
method                   object
local_site_name          object
site_address             object
state                    object
county                   object
city                     object
cbsa_code                object
cbsa    

In [32]:
# deleting unwanted columns
required_col = mongo_data.drop(['_id', 'state_code','county_code', 'site_number','parameter_code','first_max_hour', 'method_code','method',
             'poc', 'latitude','longitude', 'datum','parameter','cbsa_code', 'site_address',
             'sample_duration_code', 'sample_duration','pollutant_standard', 'units_of_measure','event_type'], axis = 1)

In [34]:
required_col.head()


,date_local,observation_count,observation_percent,validity_indicator,arithmetic_mean,first_max_value,aqi,local_site_name,state,county,city,cbsa,date_of_last_change
0,2024-01-01,24,100.0,Y,20.083333,43.0,NaN,Seattle-10th & Weller,Washington,King,Seattle,"Seattle-Tacoma-Bellevue, WA",2025-04-04
1,2024-01-01,1,100.0,Y,20.000000,20.0,71.0,Seattle-10th & Weller,Washington,King,Seattle,"Seattle-Tacoma-Bellevue, WA",2025-04-04
2,2024-01-01,1,100.0,Y,20.000000,20.0,71.0,Seattle-10th & Weller,Washington,King,Seattle,"Seattle-Tacoma-Bellevue, WA",2025-04-04
3,2024-01-01,1,100.0,Y,20.000000,20.0,71.0,Seattle-10th & Weller,Washington,King,Seattle,"Seattle-Tacoma-Bellevue, WA",2025-04-04
4,2024-01-01,1,100.0,Y,20.000000,20.0,71.0,Seattle-10th & Weller,Washington,King,Seattle,"Seattle-Tacoma-Bellevue, WA",2025-04-04


In [38]:
required_col['cbsa'] = required_col['cbsa'].fillna('Unknown')  #filling missing values in column 'cbsa'
required_col['local_site_name'] = required_col['local_site_name'].fillna('Unknown')  #filling missing values in column 'local_site_name'
required_col['aqi'] = required_col['aqi'].fillna(required_col['arithmetic_mean']) # filling missing values in aqi column with values from mean columns

In [40]:
# convert normal date format to datetime, so that year and month can be extracted
required_col['date_local'] = pd.to_datetime(required_col['date_local'], errors = 'coerce') 

In [42]:
print(required_col['date_local'].head()) # check few entries to check the format

required_col['year'] = required_col['date_local'].dt.year #extracted year from date_local column and create a new column 'year'
required_col['month'] = required_col['date_local'].dt.month #extracted month from date_local column and create a new column 'month'
required_col['week'] = required_col['date_local'].dt.isocalendar().week


0   2024-01-01
1   2024-01-01
2   2024-01-01
3   2024-01-01
4   2024-01-01
Name: date_local, dtype: datetime64[ns]


In [44]:
required_col.head()


,date_local,observation_count,observation_percent,validity_indicator,arithmetic_mean,first_max_value,aqi,local_site_name,state,county,city,cbsa,date_of_last_change,year,month,week
0,2024-01-01,24,100.0,Y,20.083333,43.0,20.083333,Seattle-10th & Weller,Washington,King,Seattle,"Seattle-Tacoma-Bellevue, WA",2025-04-04,2024,1,1
1,2024-01-01,1,100.0,Y,20.000000,20.0,71.000000,Seattle-10th & Weller,Washington,King,Seattle,"Seattle-Tacoma-Bellevue, WA",2025-04-04,2024,1,1
2,2024-01-01,1,100.0,Y,20.000000,20.0,71.000000,Seattle-10th & Weller,Washington,King,Seattle,"Seattle-Tacoma-Bellevue, WA",2025-04-04,2024,1,1
3,2024-01-01,1,100.0,Y,20.000000,20.0,71.000000,Seattle-10th & Weller,Washington,King,Seattle,"Seattle-Tacoma-Bellevue, WA",2025-04-04,2024,1,1
4,2024-01-01,1,100.0,Y,20.000000,20.0,71.000000,Seattle-10th & Weller,Washington,King,Seattle,"Seattle-Tacoma-Bellevue, WA",2025-04-04,2024,1,1


In [58]:
# maximum observation count by local aqi, local site name, city and county

obs_count = required_col.groupby(['aqi', 'local_site_name', 'city', 'county', 'date_local'])['observation_count'].max().reset_index()

# Optionally sort it by observation count to see the highest values first
obs_count = grouped.sort_values(by='observation_count', ascending=False)

print(obs_count)


              aqi                              local_site_name        city  \
0       -1.704167                             Tacoma-S 36th St      Tacoma   
5326     6.083333  Sunnyside-S16th St (Harrison Middle School)   Sunnyside   
5340     6.125000                         Bremerton-Spruce Ave   Bremerton   
5339     6.125000                        Bellingham-Pacific St  Bellingham   
5338     6.120833                        SEATTLE - BEACON HILL     Seattle   
...           ...                                          ...         ...   
10306   21.000000                               Colville-E 1St    Colville   
10305   21.000000                               Colville-E 1St    Colville   
10304   21.000000                               Colville-E 1St    Colville   
10303   21.000000                               Colville-E 1St    Colville   
15639  166.000000                            TACOMA - L STREET      Tacoma   

        county date_local  observation_count  
0       Pierce 2

In [48]:
# to get output by the couties, cities, month and week 

sorted_ = required_col.groupby(['county', 'city', 'month', 'week']).agg({
    'arithmetic_mean': 'mean',
    'first_max_value': 'max',
    'aqi': 'mean',
    'local_site_name': 'nunique',
    'year': 'first'
}).reset_index()

In [50]:
# Renaming the columns according the transformation for clarity
group_df.rename(columns={'arithmetic_mean':'weekly_pollutant_mean', 'aqi':'weekly_aqi_avg'},inplace=True)

In [56]:
# data sort
group_df.sort_values(by=['county', 'city', 'month'], inplace=True)

print(group_df)

      county       city  month  week  weekly_pollutant_mean  first_max_value  \
0      Clark  Vancouver      1     1               5.233598             20.0   
1      Clark  Vancouver      1     2               4.232879             18.0   
2      Clark  Vancouver      1     3              11.234921             37.0   
3      Clark  Vancouver      1     4               2.589792             16.0   
4      Clark  Vancouver      1     5               5.274074             17.0   
...      ...        ...    ...   ...                    ...              ...   
1027  Yakima     Yakima     12    48               9.602778             19.0   
1028  Yakima     Yakima     12    49               6.631224             16.0   
1029  Yakima     Yakima     12    50              12.554480             43.0   
1030  Yakima     Yakima     12    51              11.912441             25.0   
1031  Yakima     Yakima     12    52               6.448005             23.0   

      weekly_aqi_avg  local_site_name  

In [ ]:
group_df.to_csv('grouped_air_quality_data.csv', index=False)